<h1>Booter (Black)List [all in one]</h1> 

This is a <a href="http://jupyter.org/">Jupyther Notebook</a> that merges classes, functions and python scripts (all in one single place). Our goal is to collect an extensive list of Booter Websites. Each "cell" in this notebook is an independent part of the code towards the colection of a Booter list. For this reason we "import" needed libraries per "cell" to facilitate your understanding on the requirements. 

ANYONE can reproduce our work and generate their own Booter (black)List. We particularly use the methodology described in this notebook to generate the most extensive list of Booters (available at <a href="http://booterblacklist.com">booterblacklist.com</a>). The main difference between our list and the list generated using this methodology is that we (also) make available Booter websites that WERE online in the past but disapeared from the Internet (and therefore the crawler will not find). We also have access to ALL the registered domains in the ".com", ".net", ".org", and ".nl" Top Level Domains (TLD), which we use to classify whether a domain is a Booter (or not). Do not hesitate to <a href="mailto:j.j.santanna@utwente.nl">contact us</a> if you have any question. We would appreciate your comments and suggestions to improve our methodology. Join us!

An academic paper that describes our entire methodology is under reviewing process. Our methodology rely on two parts: (1) to collect an extensive list of URLs potentially related to Booter websites and afterwards (2) to classify the collected list in the previous part whether it is a Booter website or not. We split each part in several other subparts as following:
<br>
<br>

<div id="TOC">
<ul>
<li><a href="#1"><b>1. Defining functions to colect and store information of URLs potential related to Booters' website</b></a></li>
<ul>
    <li><a href="#1.1">1.1. Database functions:</a> defines several functions to interact with the database that stores the information of each URL potential related to a Booter; </li>
    <li><a href="#1.2">1.2. Partitioning URL(s):</a> this class extracts several attributes that are parts of a URL (input);</li>
    <li><a href="#1.3">1.3. General Scraper:</a> get general information from a Web page;</li>
    <li><a href="#1.4">1.4. Crawler & in-depth Scraper:</a> pretends to be a browser to retrieve information of webpages; collects ONLY information of potential Booter websites;</li>
    <li><a href="#1.5">1.5. Refining our Crawler/Scraper</a></li>
    <ul>
    <li><a href="#1.5.1">1.5.1. Applied to Google </a></li>
    <li><a href="#1.5.2">1.5.2. Applied to Youtube:</a> search for URLs in the description of videos found using Booter-related keywords;</li>
    <li><a href="#1.5.3">1.5.3. Applied to Hackerforum.net </a></li>
    </ul>
</ul>
<li><a href="#X"><b>!!! Instantiating the Crawlers and collecting the list of URL potentially related to Booters websites.</b></a></li>

</ul>
</div>
<br>
[ATTENTION: I'm still moving the classifier to this Notebook. If you can't wait you can see directly in the folder "Classifier".]
<br>
<br>
This research project started at the University of Twente, The Netherlands, by <a href="http://jairsantanna.com">Jair Santanna</a> with collaboration of Justyna Chromik and <a href="https://github.com/JoeyDeVries"> Joey de Vries</a>. 
<br>
<br>

<div id="1"><h1><a href="#TOC">1. Defining functions to colect and store information of URLs potential related to Booters' website</a></h1></li>

<div id="1.1"><h2><a href="#TOC">1.1. Database functions</a></h2></div>

In [1]:
import os.path
import datetime
import sqlite3

# Checking if the Database exist! If not creates a new 'Booters.db' database.
if (os.path.isfile('BOOTERS.db')) == False:
    connection = sqlite3.connect('BOOTERS.db')
    c = connection.cursor()
    c.execute("CREATE TABLE urls ( domainName,\
                                fullURL,\
                                timeUpdate,\
                                'booter?',\
                                status,\
                                srcInformation,\
                                timeAdd,\
                                classification,\
                                notes)") 
    c.execute("CREATE TABLE scores ( URL,\
                                timeUpdate,\
                                numberPages,\
                                urlType,\
                                averageDepthLevel,\
                                averageUrlLength,\
                                domainAge,\
                                domainReservationDuration,\
                                whoisPrivate,\
                                dps,\
                                pageRank,\
                                averageContentSize,\
                                outboundHyperlinks,\
                                categorySpecificDictionary,\
                                resolverIndication,\
                                termsServicesPage,\
                                loginFormDepthLevel)")
    c.execute("CREATE TABLE characteristics (URL,\
                                        timeUpdate,\
                                        numberPagesRaw,\
                                        urlTypeRaw,\
                                        averageDepthLevelRaw,\
                                        averageUrlLengthRaw,\
                                        domainAgeRaw,\
                                        domainReservationDurationRaw,\
                                        whoisPrivate,\
                                        dps,\
                                        pageRankRaw,\
                                        averageContentSizeRaw,\
                                        outboundHyperlinksRaw,\
                                        categorySpecificDictionaryRaw,\
                                        resolverIndication,\
                                        termsServicesPage,\
                                        loginFormDepthLevelRaw)")
    connection.commit()
    connection.close()

# Open connection and retrieve (single) cursor
connection = sqlite3.connect('BOOTERS.db') 

# saves a Booter URL in the database. If the Booter URL was not yet found a row is inserted,
# otherwise updated.
def SaveURL(booterURL, source = '', status='?', notes=''):
    url = booterURL.URL
    url_unique = booterURL.UniqueName()
    # Check if booter's url already exists
    if RowExists('urls', url_unique):
        # If entry exists, only do necessary updates 
        Update('urls', url_unique, 'timeUpdate', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')) # so we can see which URLS are from august 1 onwards (test set)
        # Also append source information if not yet stored
        sources = GetSingleValue('urls', url_unique, 'srcInformation')
        if source not in sources:
            Update('urls', url_unique, 'srcInformation', source if sources == '' else sources + ';' + source)
    else:
        # else, insert into database
        Insert('urls', [url_unique, url, 'CURRENT_DATE', '?', status, source, 'CURRENT_DATE', 'A', notes])

# Saves a score vector of a single Booter in the database if the score vector was not yet 
# found a row is inserted, otherwise updated.
def SaveScore(table, booterURL, 
    last_update, nr_pages, url_type, average_depth_level, average_url_length, domain_age, 
    domain_reservation_duration, whois_private, dps, page_rank, average_content_size, 
    outbound_hyperlinks, category_specific_dictionary, resolver_indication, terms_of_services_page,
    login_form_depth_level):
    url = booterURL.URL
    url_unique = booterURL.UniqueName()
    # check if booter's url already exists
    if RowExists(table, url_unique):
        # if entry exists, only do a necessary updates
        Update(table, url_unique, 'lastUpdate', last_update) 
        Update(table, url_unique, 'nr_pages', nr_pages) 
        Update(table, url_unique, 'url_type', url_type) 
        Update(table, url_unique, 'average_depth_level', average_depth_level) 
        Update(table, url_unique, 'average_url_length', average_url_length) 
        Update(table, url_unique, 'domain_age', domain_age) 
        Update(table, url_unique, 'domain_reservation_duration', domain_reservation_duration) 
        Update(table, url_unique, 'whois_private', whois_private) 
        Update(table, url_unique, 'dps', dps) 
        Update(table, url_unique, 'page_rank', page_rank) 
        Update(table, url_unique, 'average_content_size', average_content_size) 
        Update(table, url_unique, 'outbound_hyperlinks', outbound_hyperlinks) 
        Update(table, url_unique, 'category_specific_dictionary', category_specific_dictionary) 
        Update(table, url_unique, 'resolver_indication', resolver_indication) 
        Update(table, url_unique, 'terms_of_services_page', terms_of_services_page) 
        Update(table, url_unique, 'login_form_depth_level', login_form_depth_level) 
    else:
        # else, insert into database
        Insert(table, [url_unique, last_update, nr_pages, url_type, average_depth_level,
            average_url_length, domain_age, domain_reservation_duration, whois_private, dps,
            page_rank, average_content_size, outbound_hyperlinks, category_specific_dictionary,
            resolver_indication, terms_of_services_page, login_form_depth_level])	

# Checks whether a row/entry already exists by comparing a specific column with a check_value for uniqueness
def RowExists(table, url):
    query = 'SELECT domainName FROM ' + table + ' WHERE domainName = \'' + url + '\''
    cursor = connection.execute(query)
    if cursor.fetchall():
        return True
    else:
        return False

# Returns a single value of a single column from the database
def GetSingleValue(table, key_value, column, key_column = 'domainName'):
    query = 'SELECT ' + column + ' FROM ' + table + ' WHERE ' + key_column + ' = \'' + key_value + '\''
    cursor = connection.execute(query)
    return cursor.fetchone()[0]

# For easy insert statements
def Insert(table, values):
    query = 'INSERT INTO ' + table + ' VALUES ('
    for value in values:
        if value == 'CURRENT_DATE':
            query += value + ', '
        else:
            query += '\'' + str(value) + '\'' + ', '
    query = query[:-2] + ')'
    connection.execute(query)
    connection.commit()

# For easy update statements
def Update(table, key, column, value):
    query = 'UPDATE ' + table + ' SET ' + column + ' = \'' + str(value) + '\' WHERE domainName = \'' + key + '\''
    connection.execute(query)
    connection.commit()

def Select(query):
    result = []
    for row in connection.execute(query):
        result.append(row)
    return result

def CloseConnection():
    connection.close()

<div id="1.2"><h2><a href="#TOC">1.2. Partitioning URL</a></h2></div>

In [2]:
from urlparse import urlparse

# container format for potential Booter URL/domainname (PBD)
# Holds hostname/domain, complete URL and easy access to other relevant data
class BooterURL:
    # constructor
    def __init__(this, url):
        # if url does not contain protocol; add it.
        if 'http' not in url:
            url = 'http://' + url
        # parse URL and store relevant data
        parsed  = urlparse(url)
        this.Hostname = parsed.hostname
        this.Scheme   = parsed.scheme
        this.URL = parsed.scheme + '://' + parsed.netloc + parsed.path
        this.Path     = parsed.path
        this.Query    = '?' + parsed.query if parsed.query != '' else ''
        this.Full_URL = url
        this.Status	  = '?'

    # returns a URL representation that uniquely identifies the current URL 
    # this is the exact format described as a Potential Booter domain name (PBD)
    # Type 2 URLs are omitted
    def UniqueName(this):
        # here we assume the hostname to be a unique identification 
        protocol = this.Scheme + '://' if this.Scheme else ''
        if this.Hostname:
            return this.Hostname.replace('www.','')
        else:
            return ''

    def __str__(this):
        return this.UniqueName()

<div id="1.3"><h2><a href="#TOC">1.3. General Scraper</a></h2></div>

In [3]:
import requests
import cfscrape
import tldextract # https://github.com/john-kurkowski/tldextract
from lxml import html

# hosts functionality and data relevant to crawling a single web page of a domain. 
# This includes per-page properties like html content, headers and functionality to retrieve all 
# inbound and/or outbound URLs and per-category dictionary queries.
class CrawlPage:
    # constructor
    def __init__(this, response):
        this.URL     = BooterURL(response.url)
        this.HTML 	 = response.text
        this.Tree	 = html.fromstring(this.HTML)

        # create relative path for further URL queries 
        path = this.URL.Path
        if len(path) > 0 and path[0] =='/':
            this.RelativeURL = (this.URL.Hostname).replace('//', '/')			
        else:
            for i in reversed(path):
                if i == '/':
                    break
                else:
                    path = path[:-1]
            this.RelativeURL = (this.URL.Hostname + path).replace('//', '/')
        # print('relative:' + this.RelativeURL)

        # add a 'contains' list of URLs NOT to scrape
        this.Excludes = {
            '#',
            'mailto',
            '.pdf',
            '.doc',
            '.rar',
            '.zip',		
            '.png',
            '.jpeg',
            '.jpg',
            '.gif',
            '.bmp',
            '.atom',
            '.rss',	
            'skype:',
            'javascript:',
            'facebook',
            'twitter',
            '.tar.gz',
            '.exe',
            '.apk',
        }

    def URLLength(this):
        return len(this.URL.Hostname + this.URL.Path)

    def URLType(this):
        subdomains = tldextract.extract(this.URL.Hostname).subdomain
        subdomains = subdomains.split('.')
        # first remove exceptions from the list
        excludes = {
            'www',
            'ww1',
            'ww2',
            '',
        }
        for exclude in excludes:
            if exclude in subdomains:
                subdomains.remove(exclude)
        if len(subdomains) > 0: # if domain has subdomain, it is of type 2
            return 2
        else:
            return 1 # else we default to type 1. 
        # note: there is no way to check if a URL is of type 3 programatically; see discusson in thesis document

    def GetTopDomain(this):
        extract = tldextract.extract(this.URL.Hostname)
        return extract.domain + '.' + extract.suffix


    # returns all hyperlinks that point to inbound domains (relative paths = inbound)
    def GetInboundURLs(this):
        domain = this.URL.Hostname
        URLs = this.Tree.xpath('(//a)[contains(@href, "' + domain + '") or not(contains(@href, "http"))]/@href')

        urls_found = []
        result     = []
        for url in URLs:
            try:
                excluded = False
                for exclude in this.Excludes:
                    if exclude in url.lower():
                        excluded = True
                        break
                if not excluded:
                    if domain in url:
                        url = BooterURL(url)
                        url = url.Hostname + url.Path
                        if url[len(url) - 1] == '/':
                            url = url[:-1]
                        if url not in urls_found:
                            result.append(url)
                    else:
                        url = (this.RelativeURL + '/' + url).replace('//', '/')
                        if url[len(url) - 1] == '/':
                            url = url[:-1]
                        if url not in urls_found:
                            result.append(url)
                    urls_found.append(url) # to check for duplicates
            except Exception as ex:
                pass

        return result

    # returns all outbound hyperlinks 
    def GetOutboundURLs(this):
        domain = this.URL.Hostname
        URLs = this.Tree.xpath('(//a)[contains(@href, "http") and not(contains(@href, "' + domain + '"))]/@href')


        urls_found = []
        result = []
        for url in URLs:
            excluded = False
            for exclude in this.Excludes:
                if exclude in url.lower():
                    excluded = True
                    break
            if not excluded:	
                if domain in url:
                    if url not in urls_found:
                        result.append(url)
                else:		
                    if url not in urls_found:
                        result.append(url)
                urls_found.append(url) # to check for duplicates
        return result


    # returns a tokenized list of words/phrases found in the crawled page's content
    def GetContent(this):
        text_content = []
        if len(this.HTML) < 250000: # don't run xPath on too large HTML pages, takes ages (slightly bias-ed but otherwise destroys crawler times)
            content      = this.Tree.xpath('(//p)[not(contains(@style, "hidden"))]/descendant-or-self::node()/text() | (//div)[not(contains(@style, "hidden"))]//descendant-or-self::node()[not(descendant-or-self::p) and not(descendant-or-self::script) and not (descendant-or-self::style)]/text()')
            for text in content:
                # first remove irelevant characters/symbols
                remove = { '\\t', '\\r', '\\n', '&nbsp;' }
                for removeable in remove:
                    text = text.replace(removeable, '')
                # then split text by whitespace and add each entry to text_content
                text_content.append(text.split())

            # merge all lists in text_content list into one final list of words
            text_content = [item for sublist in text_content for item in sublist]
        else:
            # add bogus content to text_content (constant set as 1000)
            for i in range(0, 1000):
                text_content.append('too_large_html')

        return text_content

    # returns a bit-wise result whether this page contains an HTML login form (or register)
    def HasLoginForm(this):
        forms = this.Tree.xpath('(//form)//input[contains(@type, "password")]')
        return len(forms) > 0


    def __str__(this):
        return this.URL.Full_URL;


<div id="1.4"><h2><a href="#TOC">1.4. Crawler & in-depth Scaper</a></h2></div>

In [ ]:
import requests
import datetime
import cfscrape
import pythonwhois # https://github.com/joepie91/python-whois
import signal
from lxml import etree
from colorama import Fore, Back, Style
from random import choice, random
from time import sleep
from urlparse import urlparse

# Simple callback class for timer management
class timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

class Crawler:
    'General purpose Crawler; hosts functionality relevant to crawling a '
    'multitude of online web applications like forums, video-platforms and '
    'social media. The Crawler is not operable by itself, but acts as a '
    'superclass for specific crawler instances per web application.'
    def __init__(this, target, sleep_level=1):
        this.Target = target
        this.Sleep_Level = sleep_level
        this.URLs = []

        # This is a list of domains that we EXCLUDE from our URL-potential-Booter analysis.
        # We exclude these domains to avoid spending processing efforces as we know that 
        # subdomains of these domains CAN NOT be a Booter website. This list was composed based
        # on our observations. Therefore it can/should/may be extended in the future!!!
        this.Excludes = {
            'dropbox.com',
            'facebook.com',
            'ge.tt',
            'gyazo.com',
            'github.com',
            'hackforums.net',
            'imgur.net',
            'imgur.com',
            'mediafire.com',
            'prntscr.com',
            'pastebin.com',
            'rapidshare.com',
            'sourceforge.net',
            'twitter.com',
            'uploading.com',
            'urbandictionary.com',
            'youtube.com',
            'wikipedia',
            'wiktionary',
        }
        # heuristic phrases to determine whether a website/domain is parked i.e. for sale
        # - can be extended
        this.ParkPhrases = {
            "this domain may be for sale", 
            "this domain is for sale", 
            "buy this domain", 
            "this web page is parked", 
            "this domain name expired on", 
            "backorder this domain", 
            "this domain is available through",
        }

    # =========================================================================
    # INITIALIZATION
    # =========================================================================
    # configures all connection objects and optionally logins into the service
    def Initialize(this):
        this.PrintUpdate('initiating crawling procedures')
        this.PrintDivider()
        this.Session = requests.Session()
        # possible user-agent strings for the crawler system's 'user-agent' header flag
        # a random user_agent string is selected each subsequent crawler run as to help
        # avoid detection; manual update required from time to time
        user_agents = [ 
            # 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
            # 'Opera/9.25 (Windows NT 5.1; U; en)',
            # 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
            # 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36', # this is currently enough for all purposes
        ] 
        # http header
        this.Header = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
            'Accept-Encoding': 'gzip, deflate, sdch',
            'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
            'Cache-Control' : 'max-age=0',
            'Connection': 'keep-alive',
            'User-Agent': choice(user_agents),
        }
        this.PrintDebug(str(this.Header))


    # Follows login procedures to initiate a session object per web application
    def Login(this, url, succes_page, post_data={}):
        this.PrintDivider()
        this.PrintUpdate('attempting to login at: ' + url)
        this.PrintDivider()

        response = this.Session.post(url, data=post_data, headers=this.Header)
        js_scraper = cfscrape.create_scraper()
        # login process specific to hackforums.net; during the reseach hackforums.net enabled additional
        # security checks at their login services. Continously requesting login attemps seems to bypass
        # their detective measures. Can (and should) be generlized in the future.
        redirect_count = 1
        while 'HackForums.net has enabled additional security.' in response.text: 
            print('HackForums.net detection bypass -redirect.')
            response = this.Session.post(url, data=post_data, headers=this.Header)
            redirect_count = redirect_count + 1
            if redirect_count == 5000: # increase to 5000, seemed to work last time?
                break
        if response.url == succes_page:
            this.PrintUpdate('login succesful')
            this.PrintDivider()
            return True
        else:
            this.PrintError('failed to login to target server. Is the target online or blocked?')
            return False

    # adds a list of url substrings thst should be excluded from list of URLs
    def AddExcludes(this, excludes):
        this.Excludes = excludes

    # =========================================================================
    # CRAWLER: GENERATING A PBD LIST
    # =========================================================================
    # crawls the web service, should be overriden in each subclass
    def Crawl(this, max_date):
        this.PrintError('Crawler.Crawl function not instantiated!')

    # determines whether the url should be excluded
    def IsExcluded(this, URL):
        for excluded in this.Excludes:
            if excluded in URL.Full_URL:
                return True
        return False

    # determines whether a url is already added to the URL list
    def IsDuplicate(this, URL):
        for i in range(len(this.URLs)):
            if this.URLs[i].UniqueName() == URL.UniqueName():
                return True
        return False

    # adds a URL to the final URL/PBD list if it meets all conditions
    def AddToList(this, URL, source='?'):
        if not this.IsDuplicate(URL) and not this.IsExcluded(URL):
            try:
                # get status/respnse-code and resolved-url of URL
                status = this.GetStatus(URL.Full_URL)
                URL    = BooterURL(status[2])
                # save in database
                SaveURL(URL, source, status[0])
                # then save in list if proper response or post error
                if status[1] == 200 or status[1] == 403 or status[1] == 202:
                    BooterURL.Status = status # add status to URL for later use
                    this.URLs.append(URL)
                    this.PrintLine('ADDED TO BE SCRAPED: ' + URL.Full_URL, Fore.BLUE)
                    if status[1] == 403 or status[1] == 202:
                        print('Website blocked crawler; manually verify!')
                else:
                    this.PrintNote('incorrect response code [' + str(status[1]) + ']: ' + URL.Full_URL)
            except Exception as ex:
                this.PrintError('EXCEPTION: ' + str(ex))

    # finish crawling and output URL/PBD list to file
    def Finish(this, output_file):
        this.PrintUpdate('writing output to file \'' + output_file + '\'')
        f = open(output_file, 'w')
        for URL in this.URLs:
            f.write(URL.UniqueName() + '\n')
        this.PrintUpdate('FINISHED; closing operations')
        this.PrintDivider()
        return this.URLs

    # sleeps for a semi-random amount of time to mitigate bot detection
    def Sleep(this):
        # semi-randomly vary sleep amount to emulate human behavior
        sleep(this.Sleep_Level + random() * this.Sleep_Level) 

    # enables javascript to circumvent JS bot detection (GET the actual Web Page)
    def JSCrawl(this, url):
        js_scraper = cfscrape.create_scraper()
        response = js_scraper.get(url, headers=this.Header, timeout=15.0) 
        redirect_count = 0
        while 'http-equiv="Refresh"' in response.text: 
            response = js_scraper.get(response.url, headers=this.Header, timeout=15.0) 
            redirect_count = redirect_count + 1
            if redirect_count == 5:
                this.PrintError('REDIRECT COUNT OF 5 REACHED! ' + response.url)
                break
        return response

    # retrieves status of website: resolves URL, response code and whether it
    # is offline/online or for sale
    def GetStatus(this, url):
        response = this.JSCrawl(url)
        if response.status_code == 200 or response.status_code == 403 or response.status_code == 202:
            # check if for sale, otherwise site deemed as online
            for phrase in this.ParkPhrases:
                if phrase in response.text.lower():
                    return ('free', response.status_code, response.url)
            # else site is online
            return ('on', response.status_code, response.url)				
        else:
            return ('off', response.status_code, response.url)
    # =========================================================================
    # SCRAPER: EVIDENCE AND HEURISTICS
    # =========================================================================
    # scrapes a (potential) Booter URL for evidence as reported by Booter 
    # characteristics in 'The Generation of Booter (black)lists'
    def Scrape(this, URL, days_update=0):
        # check if number of days_update days have passed since last update, and if so, update
        if RowExists('scores', URL.UniqueName()): 
            last_update  = GetSingleValue('scores', URL.UniqueName(), 'lastUpdate')
            last_update  = datetime.datetime.strptime(last_update, '%Y-%m-%d %H:%M:%S')
            current_date = datetime.datetime.now()
            difference   = (current_date - last_update).days
            if difference < days_update:
                this.PrintDivider()
                this.PrintDebug('Skip scrape: ' + URL.UniqueName() + '; last scraped: ' + str(last_update))
                return 
        # else, start scraping
        try:
            this.PrintDivider()
            this.PrintDebug('STARTING SCRAPE: ' + URL.Full_URL)
            ### 1. structure characteristics
            this.PrintDivider()
            this.PrintUpdate('obtaining structure-based characteristics')
            this.PrintDivider()
            # - 1.1 number of pages
            number_of_pages 	= -1.0
            number_of_pages_raw = -1.0 # also store a raw score for data analysis 
            # crawl through the URL and each subsequent inbound URL 
            crawled     = []
            crawl_count = 0
            max_urls    = 50
            # - get landing page
            landing_page = CrawlPage(this.JSCrawl(URL.Full_URL))
            inbounds     = landing_page.GetInboundURLs()
            crash_pages  = []

            this.PrintNote('scraping: ' + URL.Full_URL)	
            page_url	 = (landing_page.URL.Hostname + landing_page.URL.Path).replace('//','/')
            if page_url[len(page_url) - 1] == '/':
                page_url = page_url[:-1]
            depth_levels = { page_url : 0 }
            inbounds.append(page_url)

            # 1.1.1 store depth levels of landing page found inbound urls
            for inbound in inbounds:
                if inbound not in URL.Full_URL and inbound not in page_url:
                    depth_levels[inbound] = 1

            crawled.append(landing_page)

            # 1.1.2. then from each found (inbound) URL, keep crawling until maximum crawl limit is reached
            fail_loop_attempts = 0		
            while crawl_count < len(inbounds) and crawl_count < max_urls - 1:
                inbound = inbounds[crawl_count]
                if inbound not in crawled and inbound not in crash_pages:
                    try:
                        with timeout(seconds=20):
                            # crawl next page and obtain new inbound/outbound urls
                            this.PrintNote('scraping: ' + 'http://' + inbound)

                            response = this.JSCrawl('http://' + inbound)
                            crawl_page    = CrawlPage(response)
                            new_inbounds  = crawl_page.GetInboundURLs()
                            # for each of the new_inbounds, set their depth level if less than currently stored (or not yet stored)
                            for new_inbound in new_inbounds:
                                if new_inbound not in depth_levels:
                                    depth_levels[new_inbound] = depth_levels[inbound] + 1
                            # merge results
                            inbounds  = inbounds  + list(set(new_inbounds)  - set(inbounds))
                            # then continue
                            crawled.append(crawl_page)
                            crawl_count = crawl_count + 1
                    except Exception as ex:
                        this.PrintError('EXCEPTION: ' + str(ex))
                        crash_pages.append(inbound)
                        crawl_count = crawl_count + 1
                else:
                    fail_loop_attempts += 1 # aborts loop after 10000 tries; indicating infinite loop
                    if fail_loop_attempts > 10000:
                        this.PrintError('INFINITE LOOP detected; aborting scrape')
                        break


            # 1.1.3 calculate scores
            # use quadratic equation (to give numbers with low pages higher scores)
            # equation: y = -2x^2 + 1 ... (y=0) = 0.707
            # with 25 pages, score is 0.75, so first half pages give 1/4 drop-down in score
            number_of_pages_raw = len(inbounds)
            if number_of_pages_raw > 0:
                number_of_pages = -2 * (number_of_pages_raw / (50/0.707106781	)) ** 2 + 1
                number_of_pages = max(number_of_pages, 0.0) 
            this.PrintUpdate('number of pages: ' + str(number_of_pages_raw))

            # - 1.2. URL type
            # how to determine its url type? difficult/impossible to determine programmaticaly 	
            url_type_raw = landing_page.URLType()
            if url_type_raw == 2:
                url_type = 0.0
            else:
                url_type = 1.0
            this.PrintUpdate('url type: ' + str(url_type_raw))

            # - 1.3. Average depth level
            # take previously retrieved depth levels and take average
            average_depth_level     = 0.0
            average_depth_level_raw = 0.0
            for depth_url in depth_levels:
                average_depth_level_raw = average_depth_level_raw + depth_levels[depth_url]
            # calculate score: take linear value between 1.0 and 3.0
            average_depth_level_raw = average_depth_level_raw / len(depth_levels)
            if average_depth_level_raw <= 1.0:
                average_depth_level = 1.0
            else:
                average_depth_level = max(1.0 - ((average_depth_level_raw - 1.0) / 2.0), 0.0)
            this.PrintUpdate('average depth level: ' + str(average_depth_level_raw))

            # - 1.4. Average URL length
            average_url_length     = -1.0
            average_url_length_raw = -1.0
            for page in inbounds: # use inbounds, not pages crawled as they give much more results
                average_url_length_raw = average_url_length_raw + len(page)
            # calculate score: interpolate linearly from lowest occurence to highest Booter occurence
            average_url_length_raw = average_url_length_raw / len(inbounds) 
            if average_url_length_raw <= 15:
                average_url_length = 1.0
            else:
                average_url_length = max(1.0 - ((average_url_length_raw - 15) / 15), 0.0)
            this.PrintUpdate('average url length: ' + str(average_url_length_raw))


            ### 2. content-based characteristics
            this.PrintDivider()
            this.PrintUpdate('obtaining content-based characteristics')
            this.PrintDivider()

            # get whois information
            # "Each part represents the response from a specific WHOIS server. Because the WHOIS doesn't force WHOIS 
            # servers to follow a unique response layout, each server needs its own dedicated parser."
            domain_age                      = -1.0
            domain_age_raw                  = -1.0
            domain_reservation_duration     = -1.0
            domain_reservation_duration_raw = -1.0
            try:
                with timeout(seconds=10):
                    whois = pythonwhois.get_whois(landing_page.GetTopDomain(), False) # http://cryto.net/pythonwhois/usage.html
            except Exception as ex:
                this.PrintError('EXCEPTION: get WHOIS data: ' + str(ex))
            try:
                # - 2.1. Domain age
                current_date    = datetime.datetime.today()
                date_registered = whois['creation_date'][0]
                domain_age_raw	= (current_date - date_registered).days
                # calculate score: linear interpolation between current_date and first occurence of 
                # booter in data: 2011
                days_since_first = (current_date - datetime.datetime(2011, 10, 28)).days
                domain_age = max(1.0 - (domain_age_raw / days_since_first), 0.0)
                this.PrintUpdate('domain age: ' + str(domain_age_raw))
            except Exception as ex:
                this.PrintError('EXCEPTION: whois keywords, likely registrar: ' + str(ex))

            try:
                # - 2.2 Domain reservation duration
                current_date  				    = datetime.datetime.today()
                expire_date    			        = whois['expiration_date'][0]
                domain_reservation_duration_raw = (expire_date - current_date).days
                # calculate score: between 1 - 2 years; < 1 year = 1.0
                if domain_reservation_duration_raw < 183:
                    domain_reservation_duration = 1.0
                else:
                    # domain_reservation_duration = max(1.0 - ((domain_reservation_duration_raw - 365) / 365), 0.0)
                    domain_reservation_duration = max(1.0 - (domain_reservation_duration_raw - 183) / 182, 0.0)
                this.PrintUpdate('domain reservation duration: ' + str(domain_reservation_duration_raw))
            except Exception as ex:
                this.PrintError('EXCEPTION: whois keywords, likely registrar: ' + str(ex))

            # - 2.3. WHOIS private
            # there doesn't exist a private WHOIS field, but private information can be obtained through
            # heuristics using common phrases found by privacy-replacing registry information.
            try:
                private_phrases = [
                    'whoisguard',
                    'whoisprotect',
                    'domainsbyproxy',
                    # 'whoisprivacyprotect', # are caught by privacy term anyways
                    'protecteddomainservices',
                    # 'myprivacy',
                    # 'whoisprivacycorp',
                    # 'privacyprotect',
                    'namecheap',
                    'privacy',
                    'private',
                ]
                whois_private = 0.0
                reg_name 	  = whois['contacts']['registrant']['name'].lower()
                reg_email 	  = whois['contacts']['registrant']['email'].lower()
                for phrase in private_phrases:
                    if phrase in reg_name or phrase in reg_email: #or phrase in reg_org :
                        whois_private = 1.0
                        break
            except Exception as ex:
                this.PrintError('EXCEPTION: whois keyfields, private set to -1.0: ' + str(ex))
                whois_private = -1.0
            this.PrintUpdate('WHOIS private: ' + str(whois_private))

            # - 2.4. DPS
            # similar to whois private, use heuristics to determine whether website uses DPS,
            # first we try to determine whether it uses DNS based DPS by checking nameservers
            try:
                with timeout(seconds=10):
                    dps_names = [
                        'cloudflare',
                        'incapsula',
                        'prolexic',
                        'akamai',
                        'verisign',
                        'blazingfast',
                    ]
                    dps = 0.0
                    if 'nameservers' in whois:
                        for nameserver in whois['nameservers']:
                            if dps == 0.0:
                                for dps_name in dps_names:
                                    if dps_name in nameserver.lower():
                                        dps = 1.0
                                        break
                    # if nothing found from nameservers, also check redirection history if dps redirect page was used
                    if dps < 0.5:
                        response_text = this.Session.post(URL.Full_URL, headers=this.Header, allow_redirects=False).text
                        this.PrintNote('No DPS detected from NS; checking re-direction history')
                        for dps_name in dps_names:
                            if dps_name in response_text:
                                dps = 1.0
            except Exception as ex:
                this.PrintError('EXCEPTION: dps set to -1.0: ' + str(ex))
                dps = -1.0
            this.PrintUpdate('DPS: ' + str(dps))

            # - 2.5. Page rank
            try:
                url           = 'http://data.alexa.com/data?cli=10&dat=s&url=' + URL.Hostname
                response      = this.Session.get(url)
                tree 	      = etree.XML(response.text.encode('utf-8'))
                page_rank_raw = tree.xpath('(//REACH)/@RANK')[0]
                page_rank     = 0.0
                if int(page_rank_raw) > 200000: # determined from highest booter (ipstresser.com - vdos-s.com) minus offset
                    page_rank = 1.0
            except Exception as ex:
                page_rank_raw = 25426978.0 # set to highest occuring page rank (lower than that if non-existent)
                page_rank = 1.0

            this.PrintUpdate('Page rank: ' + str(page_rank_raw))


            ### 3. host-based characteristics
            this.PrintDivider()
            this.PrintUpdate('obtaining host-based characteristics')
            this.PrintDivider()

            # - 3.1. Average content size
            average_content_size     = 0.0
            average_content_size_raw = 0.0
            crawl_contents = []
            for crawl_page in crawled:
                crawl_content = crawl_page.GetContent();
                crawl_contents.append(crawl_content)
                average_content_size_raw = average_content_size_raw + len(crawl_content)
            average_content_size_raw = average_content_size_raw / len(crawled)
            # calculte score: linear interpolation between 50 - (avg_max_booter = 250)
            if average_content_size_raw < 50:
                average_content_size = 1.0
            else:
                average_content_size = max(1.0 - (average_content_size_raw - 50) / 200, 0.0)
            this.PrintUpdate('Average content size: ' + str(average_content_size_raw))

            # - 3.2. Outbound hyperlinks
            outbound_hyperlinks     = 0.0
            outbound_hyperlinks_raw = 0.0
            for crawl_page in crawled:
                outbound_hyperlinks_raw = outbound_hyperlinks_raw + len(crawl_page.GetOutboundURLs())
            outbound_hyperlinks_raw = outbound_hyperlinks_raw / len(crawled)
            # calculate score: linear interpolation between 0 and 2
            outbound_hyperlinks = max(1.0 - outbound_hyperlinks_raw / 2.0, 0.0)
            this.PrintUpdate('Average outbound hyperlinks: ' + str(outbound_hyperlinks_raw))

            # - 3.3. Category-specific dictionary
            dictionary = [ 'stress', 'booter', 'ddos', 'powerful', 'resolver', 'price' ] # or pric, so we can also get items like pricing
            category_specific_dictionary     = 0.0
            category_specific_dictionary_raw = 0.0
            words = landing_page.GetContent()
            for item in dictionary:
                for word in words:
                    if item in word.lower():
                        category_specific_dictionary_raw = category_specific_dictionary_raw + 1
            # - now calculate percentage of these words occuring relative to total page content
            if len(words) > 0:
                category_specific_dictionary_raw = category_specific_dictionary_raw / len(words)
            else:
                category_specific_dictionary_raw = 0.0
            # calculate score: interpolate between 0.01 and 0.05             
            category_specific_dictionary = max(1.0 - ((category_specific_dictionary_raw - 0.01) / 0.04), 0.0)
            this.PrintUpdate('Category specific dictionary: ' + str(category_specific_dictionary_raw))

            # - 3.4. Resolver indication (only the landing page); perhaps extend to all pages in future version?
            resolver_indication = 0.0
            dictionary = [ 'skype' , 'xbox', 'resolve', 'cloudflare' ]
            for item in dictionary:
                for word in words:
                    if item in word.lower():
                        resolver_indication = 1.0
            this.PrintUpdate('Resolver indication: ' + str(resolver_indication))

            # - 3.5. Terms of Services page
            terms_of_services_page = 0.0
            # - check if one of the urls contains tos or terms and service
            for url in inbounds:
                url = url.lower();
                if '/tos' in url or 'terms' in url and 'service' in url:
                    terms_of_services_page = 1.0
            # - if not yet found, also check for content hints in all the pages
            tos_phrases = [
                'terms and conditions', 
                'purposes intended', 
                'you are responsible', 
                'we have the right', 
                'terms of service',
                'understand and agree',
            ]
            if terms_of_services_page < 0.5:
                for content in crawl_contents:
                    text = ' '.join(content).lower()
                    for phrase in tos_phrases:
                        if phrase in text:
                            terms_of_services_page = 1.0
                            break
                    if terms_of_services_page > 0.5:
                        break
            this.PrintUpdate('Terms of services page: ' + str(terms_of_services_page))

            # - 3.6. Login-form depth level
            # this does also take into account register forms, but that's generally expected
            # to be on the same level as login forms so not an issue
            login_form_depth_level     = -1.0
            login_form_depth_level_raw =  3.0 # set to max found in dataset if non-existent
            forms_urls = []
            for page in crawled:
                if page.HasLoginForm(): 
                    page_url = page.URL.Hostname + page.URL.Path + page.URL.Query
                    if page_url[len(page_url) - 1] == '/':
                        page_url = page_url[:-1]
                    forms_urls.append(page_url)
            min_depth = 100
            for url in forms_urls:
                for depth_url in depth_levels:
                    if depth_url == url:
                        if depth_levels[url] < min_depth:
                            min_depth = depth_levels[url]
                        break
            if min_depth != 100:
                login_form_depth_level_raw = min_depth
            # transform to score (if depth level exceeds 2, score becomes 0)
            login_form_depth_level = min(max(1.0 - min_depth * 0.5, 0.0), 1.0)
            this.PrintUpdate('Login-form depth level: ' + str(login_form_depth_level_raw))

            ### 4. Now save the results into the database
            SaveScore('scores',
                URL,
                datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                number_of_pages,
                url_type,
                average_depth_level,
                average_url_length,
                domain_age,
                domain_reservation_duration,
                whois_private,
                dps,
                page_rank,
                average_content_size,
                outbound_hyperlinks,
                category_specific_dictionary,
                resolver_indication,
                terms_of_services_page,
                login_form_depth_level
            )
            # also store raw feature data for analysis
            SaveScore('characteristics',
                URL,
                datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                number_of_pages_raw,
                url_type_raw,
                average_depth_level_raw,
                average_url_length_raw,
                domain_age_raw,
                domain_reservation_duration_raw,
                whois_private,
                dps,
                page_rank_raw,
                average_content_size_raw,
                outbound_hyperlinks_raw,
                category_specific_dictionary_raw,
                resolver_indication,
                terms_of_services_page,
                login_form_depth_level_raw
            )
            this.Sleep()

        except Exception as ex:
            this.PrintError('EXCEPTION: Scrape failed: connection host ' + str(ex))
            # raise
    # =========================================================================
    # PRINT FUNCTIONALITY
    # =========================================================================
    def PrintLine(this, text, color=Fore.WHITE, style=Style.NORMAL):
        print(color + style + text[:80] + Style.RESET_ALL)
    def PrintDivider(this):
        this.PrintLine('................................................................................', Fore.YELLOW)
    def PrintUpdate(this, text):
        this.PrintLine('SCRAPPED: ' + text, Fore.GREEN)
    def PrintError(this, text):
        this.PrintLine('[ERROR] ' + text, Fore.RED)
    def PrintDebug(this, text):
        this.PrintLine('[DEBUG] ' + text, Fore.CYAN)
    def PrintNote(this, text):
        this.PrintLine('[NOTE] ' + text, Fore.WHITE, Style.DIM)
       

<div id="1.5"><h1><a href="#TOC">1.5. Refining our Crawler/Scraper</a></h1></div>
<div id="1.5.1"><h2><a href="#TOC">1.5.1. Refining our Crawler applied to Google</a></h2></div>

In [ ]:
from lxml import html
from colorama import Fore, Back, Style
import json

# documentation: https://developers.google.com/web-search/docs/#The_Basics

class Crawler_Google2(Crawler):
    'Crawler of Google via default web requests'
    def __init__(this, sleep_level=1):
        domain = 'https://www.google.com/search?ie=UTF-8&num=100' 
        Crawler.__init__(this, domain, sleep_level)
        this.PrintNote('CRAWLING GOOGLE')
        this.PrintDivider()
        this.Initialize()
        # this.Header['referer'] = 'utwente.nl'

    # def Login(this):
        # no login
        # this.PrintError('NO LOGIN REQUIRED')

    # overrides Crawler's crawl function
    def Crawl(this, max_results=100):
        keywords = ['Booter', 'DDOSer', 'Stresser']
        
        nr_pages = int(max_results / 100)
        this.PrintUpdate('initiating crawling procedures: Google')

        for keyword in keywords:
            this.PrintDivider()
            this.PrintNote('KEYWORD: ' + keyword)
            this.PrintDivider()
            for i in range(0, nr_pages): 
                counter = 0
                # dynamically generate search query
                query =  "&q=" + keyword+ '&start=' + str(i * 100) + '&filter=0' 
                url = this.Target + query

                # read html and parse JSON
                response = this.JSCrawl(url)                 
                tree = html.fromstring(response.text) 

                urls = tree.xpath('(//div)[@class="g"]//h3[@class="r"]/a/@href')

                split   = 10
                for url in urls:
                    this.PrintNote(str(counter)+') '+url) #jjsantanna Debugging
                    try:
                        # parse url
                        if '/url?q=' in url:
                            url = url[7:].split('&sa')[0]
                        this.AddToList(BooterURL(url), 'Google')

                        if counter % split == 0:
                            this.PrintDivider()
                        counter = counter + 1
                    except Exception as ex:
                        this.PrintError('EXCEPTION: ' + str(ex))
                this.Sleep()


        this.PrintNote('DONE; '+ str(counter)+' found, but only ' + str(len(this.URLs)) + ' potential Booters')
        this.PrintDivider()
        

crawler = Crawler_Google2(1)
crawler.Crawl(100) # up to ~500 results (more is not possible by Google)
# results_google = crawler.Finish('crawl_google2.txt')

<div id="1.5.2"><h2><a href="#TOC">1.5.2. Refining our Crawler applied to Youtube</a></h2></div>

In [ ]:
import json
import re
from lxml import html
from colorama import Fore, Back, Style

class Crawler_Youtube(Crawler):
    'Crawler of Youtube via default web requests'
    def __init__(this, sleep_level=1):
        domain = 'https://www.youtube.com/results?' 
        Crawler.__init__(this, domain, sleep_level)

        this.PrintNote('CRAWLING YOUTUBE')
        this.PrintDivider()
        this.Initialize()

    # overrides Crawler's crawl function
    def Crawl(this, max_results=100):
        keywords = ['booter', 'stresser', 'ddoser']
        nr_pages = int(max_results / 10)
        
        this.PrintUpdate('initiating crawling procedures: Youtube')

        for keyword in keywords:
            this.PrintDivider()
            this.PrintNote('KEYWORD: ' + keyword)
            for i in range(0, nr_pages): 
                counter = 0
                try:
                    # dynamically generate search query
                    query =  '&search_query="' + keyword	+ '"&page=' + str(i)
                    url = this.Target + query
                    this.PrintDivider()
                    this.PrintDebug('crawling: ' + query) 
                    # read html and parse JSON
                    response = this.JSCrawl(url)
                    tree 	 = html.fromstring(response.text) 
                    split   = 10

                    urls_found = []

                    descriptions = tree.xpath('(//div)[contains(@class, "yt-lockup-description")]/descendant-or-self::*/text()')					
                    
                    for description in descriptions:
                        # check whether description certainly doesn't hold an online booter
                        if this.StopSearching(description):
                            continue

                        # find all urls in description
                        urls = re.findall('http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', description)
                        for url in urls:
                            urls_found.append(url)


                    # also check for explicit urls in descriptions
                    urls = tree.xpath('(//div)[contains(@class, "yt-lockup-description")]//a/@href')					
                    for url in urls:
                        this.PrintNote('Found: '+str(url)) ## Debuging jjsantanna
                        if url not in urls_found:
                            urls_found.append(url)
                            

                    this.PrintDivider()
                    this.PrintUpdate('obtained ' + str(len(urls_found)) + ' potential URLs; extracting...')
                    this.PrintDivider()

                    # resolve each url and add returned url to final urls
                    for url in urls_found:
                        this.AddToList(BooterURL(url), 'Youtube')
                        counter = counter + 1
                        if counter % split == 0:
                            this.PrintDivider()	

                    this.Sleep()
                except Exception as ex:
                    this.PrintError('EXCEPTION: ' + str(ex))
                    this.Sleep()

        this.PrintUpdate('DONE; found ' + str(len(this.URLs)) + ' potential Booters')
        this.PrintDivider()


    # decides to stop crawling a specific description as soon as
    # certain stop keywords are found like 'tutorial'
    def StopSearching(this, description):
        stop_words = {
            'tutorial'
            'download:'
            'gui',
            'dekstop'
        }
        text = description.lower()
        for word in stop_words:
            if word in text:
                return True

        return False

#################    
# USAGE EXAMPLE #
#################
crawler = Crawler_Youtube(1)
crawler.Crawl(100) # up to ~500 results (similar limit to Google)
crawler.Finish('crawl_youtube.txt')

<div id="1.5.3"><h2><a href="#TOC">1.5.3. Refining our Crawler applied to Hackerforums.net</a></h2></div>

In [ ]:
from lxml import html
from colorama import Fore, Back, Style

class Crawler_Hackforums(Crawler):
    'Crawler for web services of www.hackforums.net'
    def __init__(self, sleep_level=1):
        domain = 'http://www.hackforums.net/' 
        Crawler.__init__(self, domain, sleep_level)

        self.PrintNote('CRAWLING HACKFORUMS')
        self.PrintDivider()
        self.Initialize()

    # Login to hackforums.net
    # NOTE: sometimes their login procedures block automated attempts; in that case a semi-brute
    # force attempt is executed in the Crawler superclass.
    def Login(self):
        username = 'ADD YOUR USERNAME HERE!!!' 
        password = 'ADD YOUR PASSWORD HERE!!!'
        url = self.Target + 'member.php'
        post_data = {
            'username': username,
            'password': password,
            'action': 'do_login',
            'url': 'http://www.hackforums.net/index.php',
            'my_post_key': '60aae6001602ef2e0bd45033d53f53dd'
        }
        return super(Crawler_Hackforums, self).Login(
            url, 
            self.Target + 'index.php', 
            post_data
        )

    # overrides Crawler's crawl function
    def Crawl(self, max_date):
        # crawl of hackforums.net is executed in three steps:
        # 1. we retrieve all interesting forum posts
        # 2. we extract potential Booter URLs from these posts
        # 3. collect all related evidence and calculate scores 
        #    for later evaluation
        target_url = self.Target + 'forumdisplay.php?fid=232&page='
        ### step 1. retrieve all relevant forum posts
        forum_items  	 = []
        current_page 	 = 1
        max_pages 	 	 = 1
        max_date_reached = False
        self.PrintUpdate('initiating crawling procedures: HackForums')
        self.PrintDivider()

        # crawl first forum page and parse into XML tree
        response = self.Session.post(target_url + str(current_page), headers=self.Header)
        tree 	 = html.fromstring(response.text) 

        # analyze structure and get relevant properties (via XPATH)
        self.PrintUpdate('analyzing structure and retrieving Booter candidates')
        self.PrintDivider()
        max_pages = int(tree.xpath('//a[@class="pagination_last"]/text()')[0])
        # max_pages = 1 # for debug
        # now start crawling
        while current_page <= max_pages and not max_date_reached:
            self.PrintUpdate('crawling page ' + str(current_page) + '/' + str(max_pages))
            self.PrintDivider()
            # get forum items
            forum_titles = tree.xpath('//td[contains(@class,"forumdisplay_")]/div/span[1]//a[contains(@class, " subject_")]/text()')
            forum_urls   = tree.xpath('//td[contains(@class,"forumdisplay_")]/div/span[1]//a[contains(@class, " subject_")]/@href')
            forum_dates  = tree.xpath('//td[contains(@class,"forumdisplay_")]/span/text()[1]')
            # get data of each forum item
            for i in range(len(forum_titles)):
                item = ForumItem(forum_titles[i], self.Target + forum_urls[i], forum_dates[i])
                if item.IsPotentialBooter():
                    forum_items.append(item)
                    print(item)
                # check if max date is reached
                if item.Date < max_date:
                    max_date_reached = True
                    self.PrintDivider()
                    self.PrintUpdate('date limit reached; aborting...')
                    self.PrintDivider()
                    break
            # print a divider after each forum page
            self.PrintDivider()
            # get url of next page and re-iterate
            current_page = current_page + 1
            next_url     = target_url + str(current_page)
            response     = self.Session.post(next_url, headers=self.Header)
            tree         = html.fromstring(response.text)            

            if current_page <= max_pages:
                self.Sleep()
        # forum crawling is complete, print (sub)results
        self.PrintUpdate('items found: ' + str(len(forum_items)))
        self.PrintDivider()

        ### step 2. extract potential Booters from target forum posts
        self.PrintUpdate('attempting to obtain Booter URLs')
        self.PrintDivider()
        # start crawilng for each forum item
        counter = 0
        for item in forum_items:
            # parse html
            response = self.Session.post(item.URL, headers=self.Header)
            tree 	 = html.fromstring(response.text)
            url 	 = ''
            # check for URLs inside image tags
            tree_image = tree.xpath('(//tbody)[1]//div[contains(@class,"post_body")]//a[.//img and not(contains(@href, "hackforums.net")) and not(contains(@href, ".jpg") or contains(@href, ".png") or contains(@href, ".jpeg") or contains(@href, "gif"))]/@href')
            if tree_image:
                url = tree_image[0]
            else:
                # otherwise check for URL in the post's content
                tree_links = tree.xpath('(//tbody)[1]//div[contains(@class,"post_body")]//a[not(@onclick) and not(contains(@href, "hackforums.net")) and not(contains(@href, ".jpg") or contains(@href, ".png") or contains(@href, ".jpeg") or contains(@href, ".gif"))]/@href')
                if tree_links:
                    url = tree_links[0]

            # add found url to list
            if url != '':
                self.AddToList(BooterURL(url), item.URL)

            # print a divider line every 10 results (to keep things organized)
            counter = counter + 1
            if counter % 10 == 0:        
                self.PrintDivider()

        # finished, print results
        self.PrintDivider()
        self.PrintUpdate('DONE; Resolved: ' + str(len(self.URLs)) + ' Booter URLs')
        self.PrintDivider()

<div id="X"><h1><a href="#TOC">!!! Instantiating the Crawlers and collecting the list of URL potentially related to Booters websites.</a></h1></div>

In [ ]:
from urlparse import urlparse
import datetime
import json
import random

results_google 	   = []
results_hackforums = []
results_youtube	   = []

try:
	###############################################################################
	## GOOGLE V2                                                                 ##
	## ############################################################################
	crawler = Crawler_Google2(1)
	crawler.Crawl(100) # up to ~500 results (more is not possible by Google)
	results_google = crawler.Finish('potentialBooters_google.txt')

	print()
	print()
	print()

	###############################################################################
	## YOUTUBE                                                                   ##
	## ############################################################################
	crawler = Crawler_Youtube(1)
	crawler.Crawl(100) # up to ~500 results (similar limit to Google)
	results_youtube = crawler.Finish('potentialBooters_youtube.txt')

	print()
	print()
	print()

	###############################################################################
	## HACKFORUMS                                                                ##
	###############################################################################
	crawler = Crawler_Hackforums(1) # sleep level of 1
	if crawler.Login():	# from time to time it might give a 5 sec check browser period; if so, simply manually solve this by visiting hackforums.net
		print('login succesfull')
		crawler.Crawl(datetime.datetime(2015, 3, 1)) # crawl up to may 15th
		results_hackforums = crawler.Finish('potentialBooters_hackforums.txt')

	print()
	print()
	print()
except Exception as ex:
	print('GLOBAL EXCEPTION: ' + str(ex))

###############################################################################
## MERGE CRAWL RESULTS                                                       ##
###############################################################################
# final_results = []

# def IsDuplicate(booterURL):
# 	for i in range(0, len(final_results)):
# 		if booterURL.UniqueName() == final_results[i].UniqueName():
# 			return True
# 	return False		

# def AddResults(sub_results):
# 	for booterURL in sub_results:
# 		# we make the assumption the uniqueness of a Booter url is based on its
# 		# hostname only; thus we only have TYPE 1 URLs (prove this!)
# 		if not IsDuplicate(booterURL):
# 			final_results.append(booterURL)
# 		else:
# 			print('duplicate:' + booterURL.Full_URL)

# AddResults(results_google)
# AddResults(results_youtube)
# AddResults(results_hackforums)

# crawler.PrintDivider();
# crawler.PrintUpdate('completed crawling procedures; saving output to \'crawler_output.txt\'')
# with open('crawler_output.txt', 'w') as f:
	# for booterURL in final_results:
		# f.write(booterURL.UniqueName() + '\n')
		# json.dump(vars(booterURL), f)
		# f.write('\n')

###############################################################################
## SCRAPE AND GENERATE SCORES                                                ##
###############################################################################
crawler.PrintDivider();
crawler.PrintUpdate('INITIATING SCRAPING PROCEDURES;')
crawler.PrintDivider();

# query all to-scrape URLs
# from_date    = datetime.datetime(2015, 8, 1).strftime('%Y-%m-%d %H:%M:%S') # test_scores
# from_date    = datetime.datetime(2015, 8, 19).strftime('%Y-%m-%d %H:%M:%S') #test_scores2
from_date    = datetime.datetime(2015, 8, 20, 13, 30).strftime('%Y-%m-%d %H:%M:%S') #test_scores3
delay_period = 7

for url in Select('SELECT fullURL FROM urls WHERE status != \'off\' AND timeUpdate >= \'' + str(from_date) + '\''):
	delay = delay_period + random.randint(0,14) # add a slight randomness to delay_period as to divide workload
	delay = 1
	crawler.Scrape(BooterURL(url[0]), delay)

crawler.PrintDivider();
crawler.PrintUpdate('DONE;')
crawler.PrintDivider();